# Init Notebook

In [1]:
# Import necessary libraries
from dotenv import load_dotenv
from openai import OpenAI
import os
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm

from embeddings_comparison.utils.llm_clients.cached_client import CachedLLMClient
from embeddings_comparison.utils.llm_clients.open_ai_client import OPEN_AI_MODEL_NAME, OpenAIClient


# Load environment variables from .env file
load_dotenv()

open_ai_key = os.environ["OPEN_AI_KEY"]

openai_client = OpenAIClient(api_key=open_ai_key, model_name=OPEN_AI_MODEL_NAME.GPT_4O)
cached_client = CachedLLMClient(client=openai_client)

/home/pigaand/.cache/pypoetry/virtualenvs/embeddings-comparison-JqTXriCt-py3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Data curation
Downloading set of articles to be used for assessment

In [2]:
from embeddings_comparison.utils import wiki_parser
from pprint import pprint
from embeddings_comparison.utils.storage import ArticleStorage

storage = ArticleStorage()

raw_pages = wiki_parser.extract_pages_from_file("data/Wikipedia-20241111162837.xml")
storage.save_articles(raw_pages)

# Data filtering

Remove sections that don't contain text or are not relevant

In [3]:
pages_df = storage.load_all()
pages_df = pages_df[pages_df['Section Title'] != 'Linki zewnętrzne']
pages_df = pages_df[pages_df['Section Title'] != 'Zobacz też']
pages_df = pages_df[pages_df['Section Title'] != 'Bibliografia']
pages_df

,Article Title,Section Title,Section Content
4,Drużyna sportowa,Main,Drużyna sportowa – zespół składający się z okr...
8,ASInfo,Main,"ASInfo – pierwsza w Polsce, sportowa agencja i..."
11,Kategoria:Sport według państw,Main,Do kategorii dodano również zagadnienia dotycz...
13,Duchowość w sporcie,Duchowość a sport – badania,Na Muzułmańskim Uniwersytecie w Aligarh S. T. ...
14,Sport zawodowy,Main,Sport zawodowy – oznacza uprawianie sportu w t...
15,Sport,Main,"Sport – wszelkie formy aktywności fizycznej, k..."
18,Kategoria:Jazda na rowerze,Main,"Jazda rowerem to sposób transportu, rekreacji,..."


# Augment data

In [4]:
from embeddings_comparison.utils.question_generation import BASE_PROMT_PL, generate_question_for_text
from tqdm.autonotebook import tqdm

pbar = tqdm(total=len(pages_df))
questions_column = []
for _, row in pages_df.iterrows():
    pbar.set_description(f"Generating questions for {row['Section Title']}")
    questions = generate_question_for_text(cached_client, row['Section Content'], BASE_PROMT_PL)
    questions_column.append(questions.questions)
    pbar.update(1)

pages_df['questions'] = questions_column

  0%|          | 0/7 [00:00<?, ?it/s]

# Data Ingestion

In [5]:
from embeddings_comparison.utils.embedding_models.caching import CachedEmbeddingModel
from embeddings_comparison.utils.embedding_models.hugging_face import HF_EMBEDDING_MODEL_NAME, HFEmbeddingModel
from embeddings_comparison.utils.embedding_models.open_ai import OPENAI_EMBEDDING_MODEL_NAME, OpenAIEmbeddingModel
from embeddings_comparison.utils.vectordb.vectordb import VectorDB, VectorIndex
from tqdm.autonotebook import tqdm

vector_db = VectorDB()
vector_db.add_index(f"OPENAI_SMALL", CachedEmbeddingModel(OpenAIEmbeddingModel(api_key=open_ai_key, model=OPENAI_EMBEDDING_MODEL_NAME.TEXT_EMBEDDING_3_SMALL)))
vector_db.add_index(f"OPENAI_LARGE", CachedEmbeddingModel(OpenAIEmbeddingModel(api_key=open_ai_key, model=OPENAI_EMBEDDING_MODEL_NAME.TEXT_EMBEDDING_3_LARGE)))
vector_db.add_index(f"OPENAI__ADA", CachedEmbeddingModel(OpenAIEmbeddingModel(api_key=open_ai_key, model=OPENAI_EMBEDDING_MODEL_NAME.TEXT_EMBEDDING_ADA_002)))
vector_db.add_index("HF_SDADAS", CachedEmbeddingModel(HFEmbeddingModel(model_name=HF_EMBEDDING_MODEL_NAME.ST_POLISH_PARAPHRASE_FROM_DISTILROBERTA)))

for index_name in tqdm(vector_db.list_indices(), desc='Testing Embedding models'):
    vector_db.insert_texts(pages_df['Section Content'].values.tolist(), index_name)

Testing Embedding models:   0%|          | 0/4 [00:00<?, ?it/s]

# Perform the experiment

In [6]:
from tqdm.autonotebook import tqdm

pbar = tqdm(total=len(vector_db.list_indices()))
pbar.update(1)
for index_name in vector_db.list_indices():
    pbar.set_description(f"Testing {index_name}")
    identified_matches = []
    for _, row in tqdm(pages_df.iterrows(), desc=f"Testing rows for {index_name}"):
        matched_with_answer = 0
        for question in row['questions']:
            found_text = vector_db.find_text(text=question, top_k=1, index_name=index_name)[0]

            if found_text == row['Section Content']:
                matched_with_answer += 1
        identified_matches.append(matched_with_answer)
    pages_df[index_name] = identified_matches
    pbar.update(1)

  0%|          | 0/4 [00:00<?, ?it/s]

Testing rows for OPENAI_SMALL: 0it [00:00, ?it/s]

Testing rows for OPENAI_LARGE: 0it [00:00, ?it/s]

Testing rows for OPENAI__ADA: 0it [00:00, ?it/s]

Testing rows for HF_SDADAS: 0it [00:00, ?it/s]

In [7]:
pages_df

,Article Title,Section Title,Section Content,questions,OPENAI_SMALL,OPENAI_LARGE,OPENAI__ADA,HF_SDADAS
4,Drużyna sportowa,Main,Drużyna sportowa – zespół składający się z okr...,"[Jak zdefiniować drużynę sportową?, Z jakiej l...",5,5,5,5
8,ASInfo,Main,"ASInfo – pierwsza w Polsce, sportowa agencja i...",[Kto założył pierwszą w Polsce sportową agencj...,5,5,5,5
11,Kategoria:Sport według państw,Main,Do kategorii dodano również zagadnienia dotycz...,"[Jakie nowe zagadnienia dodano do kategorii?, ...",4,5,5,5
13,Duchowość w sporcie,Duchowość a sport – badania,Na Muzułmańskim Uniwersytecie w Aligarh S. T. ...,[Jakie badanie przeprowadzono na Muzułmańskim ...,5,5,5,5
14,Sport zawodowy,Main,Sport zawodowy – oznacza uprawianie sportu w t...,[Jakie są kryteria kwalifikowania sportu jako ...,4,5,5,5
15,Sport,Main,"Sport – wszelkie formy aktywności fizycznej, k...",[Jakie są różne formy aktywności uważane za sp...,5,5,4,5
18,Kategoria:Jazda na rowerze,Main,"Jazda rowerem to sposób transportu, rekreacji,...",[Jakie rodzaje rowerów są uwzględnione w jeźdz...,5,5,5,5
